In [1]:
from bs4 import BeautifulSoup
import requests
import time
import re
import numpy as np
import pandas as pd

## loading auction list

In [2]:
auction_list = []

In [3]:
with open('apartments_links.txt','r') as file:
    for line in file:
        auction_list.append(line[:-1])

## collecting auction details and batch saving them

In [4]:
start = time.time()

i = 0 # <--- change after crash
data = []

file_name = 'apartment_details.csv'

# only for first search vvvvv
f = open(file_name, "w", encoding='utf-8')
f.write('city,district,voivodeship,localization_y,localization_x,market,offer_type,area,rooms,floor,floors,build_yr,price,url\n')
f.close()
# only for first search ^^^^^

for url in auction_list[i:]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
    except:
        continue
    # location
    try:
        location = soup.find_all('span',class_="offerLocation__region")
        city = location[0].text.strip().replace(',','')
    except:
        city = ''
    try:
        if len(location) == 3:
            district = location[1].text.strip().replace(',','')
        else:
            district = ''
    except:
        district = ''
    try:
        voivodeship = location[-1].text.strip().replace(',','')
    except:
        voivodeship = ''
    # coordinates
    try:
        scripts = soup.find(id='leftColumn').find_all('script')
        localization_y = re.findall(r'(\"\w+szerokosc.*?\"):(-?\d*[.\d]*)', scripts[4].contents[0])[0][1]
    except:
            try:
                localization_y = re.findall(r'(\"\w+szerokosc.*?\"):(-?\d*[.\d]*)', scripts[5].contents[0])[0][1]
            except:
                localization_y = ''
    try:
        localization_x = re.findall(r'(\"\w+dlugosc.*?\"):(-?\d*[.\d]*)', scripts[4].contents[0])[0][1]
    except:
            try:
                localization_x = re.findall(r'(\"\w+dlugosc.*?\"):(-?\d*[.\d]*)', scripts[5].contents[0])[0][1]
            except:
                localization_x = ''
    # parameters
    try:
        scripts2 = soup.find('body').find_all('script')
        market = re.findall(r'(\"rynek\"):\"(.*?)\"', scripts2[18].contents[0])[0][1]
    except:
        market = ''  
    try:
        offer_type = re.findall(r'(\"typ_oferty\"):\"(.*?)\"', scripts2[18].contents[0])[0][1]
    except:
        offer_type = ''
    try:
        parameters = soup.find_all('ul',class_="parameters__singleParameters")
        area = re.findall(r'(Powierzchnia).*\n(.*)m', parameters[0].text)
        area = area[0][1].replace(' ','')
        area = area.replace(',','.')
    except:
        area = ''
    try:
        rooms = re.findall(r'(Liczba pokoi).*\n(.*)', parameters[0].text)
        rooms = rooms[0][1]
    except:
        rooms = ''
    try:
        floor = re.findall(r'(Piętro).*\n(.*)', parameters[0].text)
        floor = floor[0][1]
    except:
        floor = ''
    try:
        floors = re.findall(r'(Liczba pięter w budynku).*\n(.*)', parameters[0].text)
        floors = floors[0][1]
    except:
        floors = ''
    try:
        build_yr = re.findall(r'(Rok budowy).*\n(.*)', parameters[0].text)
        build_yr = build_yr[0][1]
    except:
        build_yr = ''                   
    try:
        price_info = soup.find_all('span',class_="priceInfo__value")
        price = price_info[0].text.replace(' ','').replace('\n','').replace(',','.')[:-2]
    except:
        price = ''

    # data collection
    data.append([city,
                district,
                voivodeship,
                localization_y,
                localization_x,
                market,
                offer_type,
                area,
                rooms,
                floor,
                floors,
                build_yr,
                price,
                url])
    
    i += 1
    if i%500 == 0 or i == len(auction_list):
        print(round(i/len(auction_list)*100,2),'%', time.strftime("%H:%M:%S", time.localtime()))
        f = open(file_name, "a", encoding='utf-8')
        for line in data:
            to_write = ','.join(line) + '\n'
            f.write(to_write)
        f.close()
        data = []
        
#     time.sleep(0.5)
        
stop = time.time()
print((stop-start)//60,'min.')

0.51 % 00:26:11
1.02 % 00:34:23
1.53 % 00:42:48
2.04 % 00:50:59
2.55 % 00:59:07
3.06 % 01:07:12
3.57 % 01:15:15
4.07 % 01:23:19
4.58 % 01:31:16
5.09 % 01:39:27
5.6 % 01:47:32
6.11 % 01:55:29
6.62 % 02:03:31
7.13 % 02:11:27
7.64 % 02:19:26
8.15 % 02:27:21
8.66 % 02:35:15
9.17 % 02:43:14
9.68 % 02:51:09
10.19 % 02:59:09
10.7 % 03:07:34
11.21 % 03:15:32
11.71 % 03:23:23
12.22 % 03:31:14
12.73 % 03:39:04
13.24 % 03:46:51
13.75 % 03:55:20
14.26 % 04:03:33
14.77 % 04:11:58
15.28 % 04:22:01
15.79 % 04:32:00
16.3 % 04:41:20
16.81 % 04:50:29
17.32 % 04:59:17
17.83 % 05:07:02
18.34 % 05:15:14
18.85 % 05:23:26
19.35 % 05:31:40
19.86 % 05:39:30
20.37 % 05:47:18
20.88 % 05:55:20
21.39 % 06:03:31
21.9 % 06:12:12
22.41 % 06:20:35
22.92 % 06:28:47
23.43 % 06:36:44
23.94 % 06:44:40
24.45 % 06:52:42
24.96 % 07:02:16
25.47 % 07:12:23
25.98 % 07:22:15
26.49 % 07:32:04
27.0 % 07:42:17
27.5 % 07:52:26
28.01 % 08:03:50
28.52 % 08:14:32
29.03 % 08:26:02
29.54 % 08:37:35
30.05 % 08:48:32
30.56 % 08:59:32
31.07